<a href="https://colab.research.google.com/github/aime-n/ic/blob/main/exemplo_did%C3%A1tico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KG proposto para App Reviews

- app_id <-> snippet (relation='contain_snippet')
- app_id <-> category (relation='is_category')
- expressao1 <-> expressao2 (relation='relation') # OBS: vem do get_relation / get_entity
- snippet <-> requirement (relation='is_requirement')
- requirement <-> entity_group (relation='contain_tag')


OBS:
- Snippet é uma sentença da review.
- Ainda não temos a categoria, entao ignorar
- entity_group é uma maneira de agrupar os requisitos. Por exemplo, vários requisitos podem ser linkados com o entity_group = 'security'. Ainda não temos essa info, podemos ignorar.



In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 78 kB 3.2 MB/s 
     |████████████████████████████████| 3.5 MB 9.0 MB/s 
     |████████████████████████████████| 6.8 MB 32.1 MB/s 
     |████████████████████████████████| 1.2 MB 46.9 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 54.8 MB/s 
     |████████████████████████████████| 895 kB 46.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=c0a7f71cf8b40eeacc7acc114f69f442653bbb5480cfc392079cf09fe43cb9ae
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# KG atual para testes preliminares

- expressao1 <-> expressao2 (relation='relation') # OBS: vem do get_relation / get_entity
- snippet <-> requirement (relation='is_requirement')

In [ ]:
!git clone https://github.com/HaoWeiHe/Knowledge-Graph

Cloning into 'Knowledge-Graph'...
remote: Enumerating objects: 428, done.
remote: Counting objects: 100% (428/428), done.
remote: Compressing objects: 100% (392/392), done.
remote: Total 428 (delta 242), reused 97 (delta 31), pack-reused 0
Receiving objects: 100% (428/428), 7.67 MiB | 12.87 MiB/s, done.
Resolving deltas: 100% (242/242), done.


In [ ]:
mv ./Knowledge-Graph/* .

In [ ]:
from knowledgeGraph import get_entity
from knowledgeGraph import get_relation

In [ ]:
!pip install -q spacy==3.2

     |████████████████████████████████| 6.0 MB 3.6 MB/s 
     |████████████████████████████████| 451 kB 47.8 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 10.1 MB 39.9 MB/s 
     |████████████████████████████████| 181 kB 52.4 MB/s 
     |████████████████████████████████| 628 kB 49.2 MB/s 


In [ ]:
import pandas as pd
ground_truth = pd.read_csv('https://raw.githubusercontent.com/aime-n/ic/main/Ground_truth.csv')
#ground_truth = ground_truth.iloc[0:15]
ground_truth

,App id,Review id,Sentence id,Sentence content,Feature (Positive),Feature (Neutral),Feature (Negative),Feature (All Annotated)
0,com.zentertain.photoeditor,gp:AOqpTOEW40L9WXqCjzq04bqaZImgMdzlczxIF3_ibs8...,1,May be i can check,NaN,NaN,NaN,NaN
1,com.zentertain.photoeditor,gp:AOqpTOF57AQPvmnCiWYurwLY-F2-mej25ON8RAFk-Ls...,1,It make me happy,NaN,NaN,NaN,NaN
2,com.zentertain.photoeditor,gp:AOqpTOHYdmt72q4tSD8TZ8A5fZQqGivlBkIbWuHuJMZ...,1,I have a normal phone and it made 1 of my pics...,NaN,pics,NaN,pics
3,com.zentertain.photoeditor,gp:AOqpTOFYnXMShrDJPS0jpM04pFQxYOJN1LDuX3lSNm0...,1,Love it so much,NaN,NaN,NaN,NaN
4,com.zentertain.photoeditor,gp:AOqpTOF_JO496wnThQ2kcYlPct_g1GhOmQyyVvHp4VV...,1,Cant get to install,NaN,install,NaN,install
...,...,...,...,...,...,...,...,...
2057,com.spotify.music,gp:AOqpTOE-vqukBoo4GbnnTJnBesSlsJR9w2yGydeMlIK...,1,Every time I go on I can't browse or do anythi...,NaN,NaN,NaN,NaN
2058,com.spotify.music,gp:AOqpTOE-vqukBoo4GbnnTJnBesSlsJR9w2yGydeMlIK...,2,I am fully connected to wifi and offline mode ...,NaN,offline mode,NaN,offline mode
2059,com.spotify.music,gp:AOqpTOE-vqukBoo4GbnnTJnBesSlsJR9w2yGydeMlIK...,3,"Loved this app when it worked, can't use at al...",NaN,NaN,NaN,NaN
2060,com.spotify.music,gp:AOqpTOE-vqukBoo4GbnnTJnBesSlsJR9w2yGydeMlIK...,4,Will cancel if not fixed very soon...,NaN,NaN,NaN,NaN


In [ ]:
import nltk
from nltk.tokenize import sent_tokenize # sentence tokenizer
nltk.download('punkt')
import networkx as nx
from tqdm.notebook import tqdm

KG = nx.Graph()
source = []
target = []
index_l = []
relation_names = []


i=0
for review in tqdm(ground_truth['Sentence content']):
  for sentence in sent_tokenize(review):
    #print('-->',sentence)
    nodes = get_entity(sentence)
    #print('nodes',nodes)
    node1, node2 = nodes[0], nodes[1]

    relation = get_relation(sentence)

    #print(node1,node2,relation)

    if len(node1) > 2 and len(node2) > 2:
      index_l.append(i)
      KG.add_edge(node1,node2,relation=relation)
      source.append(node1)
      target.append(node2)
      relation_names.append(relation)
  i+=1

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


  0%|          | 0/2062 [00:00<?, ?it/s]

In [ ]:
# resultado de len(node)>0
print(f'length index_l: {len(index_l)}')
print(f'length KG.edges: {len(KG.edges)}')

In [ ]:
df_edges_linguistic = pd.DataFrame({'source': source, 'target': target, 'relation': relation_names})
df_edges_linguistic

# Adicionando relacoes snippet <=> requirement

In [ ]:
def isNaN(num):
    return num != num

for index,row in tqdm(ground_truth.iterrows(),total=len(ground_truth)):
  node1 = row['Sentence content']
  node2 = row['Feature (All Annotated)']
  if isNaN(node2)==False:
    for node in node2.split(';'):
      print(node)
      KG.add_edge(node1,node,relation='is_requirement')

In [ ]:
for i in KG.edges(data=True):
  print(i)

# Verificando todas as conexoes da estrutura linguistica

In [ ]:
for v in KG.edges(data=True):
  if v[2]['relation']!='is_requirement':
    print(v)

# Verificando conexoes entre o KG linguistico e KG requisitos anotados

In [ ]:
for v in KG.edges(data=True):
  if v[2]['relation']!='is_requirement':
    print(v)
    for v2 in KG.neighbors(v[0]):
      print("\t\t-->",v2)
    for v2 in KG.neighbors(v[1]):
      print("\t\t-->",v2)

# OBS: os requisitos extraídos pelo BERT podem ser adicionados na rede da mesma forma dos anotados manualmente...

In [ ]:
# def isNaN(num):
#     return num != num

# rebert_requisitos = CSV_REBERT
# for index,row in tqdm(rebert_requisitos.iterrows(),total=len(rebert_requisitos)):
#   node1 = row['Sentence content']
#   node2 = row['Feature (bert)'] # VERIFICAR
#   if isNaN(node2)==False:
#     for node in node2.split(';'):
#       print(node)
#       KG.add_edge(node1,node,relation='is_requirement')

# GCN

In [ ]:
# install StellarGraph if running on Google Colab
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.2.1

In [ ]:
adj = nx.adjacency_matrix(KG).todense()
g_attr = KG.copy()

c = 0
for node_id, node_data in g_attr.nodes(data = True):
  matrix_row = adj[c]
  node_data["feature"] = matrix_row
  c += 1

In [ ]:
list_nodes = list(g_attr.nodes)

matrix_adj = pd.DataFrame(adj)
matrix_adj['index'] = list_nodes
matrix_adj = matrix_adj.set_index('index')

matrix_adj.columns = list_nodes
matrix_adj.head(3)

In [ ]:
import stellargraph as sg

In [ ]:
G = sg.StellarGraph(matrix_adj, df_edges_linguistic[['source', 'target']])

In [ ]:
print(G.info())

In [ ]:
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import FullBatchLinkGenerator
from stellargraph.layer import GCN, LinkEmbedding


from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML
%matplotlib inline

In [ ]:
# Define an edge splitter on the original graph G:
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G, and obtain the
# reduced graph G_test with the sampled links removed:
G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=False)

In [ ]:
# Define an edge splitter on the reduced graph G_test:
edge_splitter_train = EdgeSplitter(G_test)

# Randomly sample a fraction p=0.5 of all positive links, and same number of negative links, from G_test, and obtain the
# reduced graph G_train with the sampled links removed:
G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
    p=0.1, method="global", keep_connected=False
)

In [ ]:
epochs = 50

In [ ]:
train_gen = FullBatchLinkGenerator(G_train, method="gcn")
train_flow = train_gen.flow(edge_ids_train, edge_labels_train)

In [ ]:
test_gen = FullBatchLinkGenerator(G_test, method="gcn")
test_flow = train_gen.flow(edge_ids_test, edge_labels_test)

In [ ]:
gcn = GCN(
    layer_sizes=[16, 16], activations=["relu", "relu"], generator=train_gen, dropout=0.3
)

In [ ]:
x_inp, x_out = gcn.in_out_tensors()

In [ ]:
prediction = LinkEmbedding(activation="relu", method="ip")(x_out)

In [ ]:
prediction = keras.layers.Reshape((-1,))(prediction)

In [ ]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=0.01),
    loss=keras.losses.binary_crossentropy,
    metrics=["binary_accuracy"],
)

In [ ]:
init_train_metrics = model.evaluate(train_flow)
init_test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

In [ ]:
history = model.fit(
    train_flow, epochs=epochs, validation_data=test_flow, verbose=2, shuffle=False
)

In [ ]:
sg.utils.plot_history(history)

In [ ]:
train_metrics = model.evaluate(train_flow)
test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

# Exemplo da Proposta: embutir features de um modelo de linguagem

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
# adj = nx.adjacency_matrix(KG).todense()
g_attr = KG.copy()

for node_id, node_data in g_attr.nodes(data = True):
  print('Gerando features para:',node_id)
  feature_linguagem =  model.encode(node_id)
  print(feature_linguagem)
  node_data["feature"] = feature_linguagem

In [ ]:
G = sg.StellarGraph.from_networkx(g_attr, node_features="feature")

In [ ]:
print(G.info())

In [ ]:
# Define an edge splitter on the original graph G:
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G, and obtain the
# reduced graph G_test with the sampled links removed:
G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=False)

In [ ]:
# Define an edge splitter on the reduced graph G_test:
edge_splitter_train = EdgeSplitter(G_test)

# Randomly sample a fraction p=0.5 of all positive links, and same number of negative links, from G_test, and obtain the
# reduced graph G_train with the sampled links removed:
G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
    p=0.1, method="global", keep_connected=False
)

In [ ]:
epochs = 50

In [ ]:
train_gen = FullBatchLinkGenerator(G_train, method="gcn")
train_flow = train_gen.flow(edge_ids_train, edge_labels_train)

In [ ]:
test_gen = FullBatchLinkGenerator(G_test, method="gcn")
test_flow = train_gen.flow(edge_ids_test, edge_labels_test)

In [ ]:
gcn = GCN(
    layer_sizes=[16, 16], activations=["relu", "relu"], generator=train_gen, dropout=0.3
)


In [ ]:
x_inp, x_out = gcn.in_out_tensors()

In [ ]:
prediction = LinkEmbedding(activation="relu", method="ip")(x_out)

In [ ]:
prediction = keras.layers.Reshape((-1,))(prediction)

In [ ]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=0.01),
    loss=keras.losses.binary_crossentropy,
    metrics=["binary_accuracy"],
)

In [ ]:
init_train_metrics = model.evaluate(train_flow)
init_test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

In [ ]:
history = model.fit(
    train_flow, epochs=epochs, validation_data=test_flow, verbose=2, shuffle=False
)

In [ ]:
sg.utils.plot_history(history)

In [ ]:
train_metrics = model.evaluate(train_flow)
test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))